<center><a href="https://www.pieriantraining.com/" ><img src="../PTCenteredPurple.png" alt="Pierian Training Logo" /></a></center>


# Function Calling with Assistants

There may custom functionality that can't be done by the LLM Assistant, but the LLM assistant is "smart" enough to know how to fill in a Python function for the task. This is where we can provide a custom function that is run on our end and work in conjunction with the assistant. This is a powerful tool, because you now have the power of the OpenAI LLM but with the full flexibility of whatever Python code you can run on your end. Let's work through an example:

## Step 1: Define your custom function

In [1]:
def word_definition_quiz(word,definition_options):
    '''
    INPUTS:
        word str = A single string representing a word
        definition_options list[str] = A list of 4 potential defintions, where only one is correct!
    OUTPUTS:
        response str = The user selected definition
    '''
    print("Hello! Let's test your knowledge about words.")
    print(f"What is the correct definition of this word: {word}\n")

    for num,option in enumerate(definition_options):
        print('\n')
        print(f"Definition #{num} is: {option}")

    print('\n')
    num_choice = input("What is your choice? (Return the single number option)")  # User has to input option

    return definition_options[int(num_choice)]

**Test the function.**

In [4]:
word = "serendipity"
definition_options = [
    "The occurrence of events by chance in a happy or beneficial way.",
    "A feeling of intense fear, shock, or disgust.",
    "A method of painting using opaque pigments ground in water and thickened with a glue-like substance.",
    "The action of delaying or postponing something."
]

# Call the function with these arguments
response = word_definition_quiz(word, definition_options)


Hello! Let's test your knowledge about words.
What is the correct definition of this word: serendipity



Definition #0 is: The occurrence of events by chance in a happy or beneficial way.


Definition #1 is: A feeling of intense fear, shock, or disgust.


Definition #2 is: A method of painting using opaque pigments ground in water and thickened with a glue-like substance.


Definition #3 is: The action of delaying or postponing something.


What is your choice? (Return the single number option)0


In [5]:
response

'The occurrence of events by chance in a happy or beneficial way.'

## Step 2: Define Custom Function in JSON Format.

Here is the example format that the Python function must be defined in, the most up to date information can be found in the OpenAI Documentation: https://platform.openai.com/docs/assistants/tools/function-calling.


**JSON TOOLS DEFINITION GENERAL FORMAT:**

    tool = {'type':'function',
                'function':{
                    'name': 'my_function_name',
                    'parameters':{
                        "type":"object",
                          "properties":{
                              "parameter_one": {'type':'string','description':"A text description for the LLM of what this parameter should be. Note the JSON type."},
                              "parameter_two":{'type':'integer','description':"A text description for the LLM of this parameter, note the JSON type."}
                          },
                        'required' : ['parameter_one','parameter_two']
                    }

        }
    }

In [ ]:
# tool = {'type':'function',
#             'function':{
#                 'name': 'my_function_name',
#                 'parameters':{
#                     "type":"object",
#                       "properties":{
#                           "parameter_one": {'type':'string','description':"A text description for the LLM of what this parameter should be. Note the JSON type."},
#                           "parameter_two":{'type':'integer','description':"A text description for the LLM of this parameter, note the JSON type."}
#                       },
#                     'required' : ['parameter_one','parameter_two']
#                 }

#     }
# }

In [18]:
# https://community.openai.com/t/function-calling-passing-a-list-of-values/369032
function_json = {'type':'function',
            'function':{
                'name': 'word_definition_quiz',
                'parameters':{
                    "type":"object",
                      "properties":{
                          "word": {'type':'string','description':"A single word"},
                          "definitions_list":{'type':'array',
                                        'items':{'type':'string'},  #As its array, we need to mention type of value/items in list
                                        'description':"A Python list of strings of definitions, with only one correct definition pertaining to the 'word' parameter. Just the string definitions."}
                      },
                    'required' : ["word","definitions_list"]
                }

    }
}

## Step 3: Create Assistant with Function Calling Tool

In [7]:
from openai import OpenAI
import os
from google.colab import userdata

OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')
os.environ['OPENAI_API_KEY'] = OPENAI_API_KEY

client = OpenAI()

In [19]:
assistant = client.beta.assistants.create(
  instructions="You help create a quiz for checking defintions of words, providing a word and then multiple definition options, where only one option is correct.",
  model="gpt-3.5-turbo",
  tools=[function_json]
)

## Step 4: Create a Thread, Message, and Run
- Here assistance will use some randm word and provides list of defination list

In [20]:
thread = client.beta.threads.create()

In [21]:
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="Create a new quiz question word and definition list. Then let me know if the student answer recieved is correct.",
)

In [22]:
run = client.beta.threads.runs.create(
  thread_id=thread.id,
  assistant_id=assistant.id)

In [12]:
import time

def wait_on_run(run, thread):
    while run.status == "queued" or run.status == "in_progress":
        run = client.beta.threads.runs.retrieve(
            thread_id=thread.id,
            run_id=run.id,
        )
        time.sleep(0.5)
    return run

In [23]:
run = wait_on_run(run,thread)

In [24]:
run.status

'requires_action'

## Step 5: Take Run Results and Call Function Locally

In [25]:
run.required_action.submit_tool_outputs.tool_calls[0]
# Here assistance created use some randm word ephemeral and provides list of defination list

RequiredActionFunctionToolCall(id='call_uWrb5uusDjADyQO1Uh88SwBk', function=Function(arguments='{"word":"ephemeral","definitions_list":["lasting for a very short time","lasting for a long period of time","occurring frequently","having a strong scent"]}', name='word_definition_quiz'), type='function')

In [26]:
import json

# Extract single tool call
tool_call = run.required_action.submit_tool_outputs.tool_calls[0]
name = tool_call.function.name
arguments = json.loads(tool_call.function.arguments)

print("Function Name:", name)
print("Function Arguments:")
arguments

Function Name: word_definition_quiz
Function Arguments:


{'word': 'ephemeral',
 'definitions_list': ['lasting for a very short time',
  'lasting for a long period of time',
  'occurring frequently',
  'having a strong scent']}

In [27]:
answer = word_definition_quiz(arguments['word'],arguments['definitions_list'])

Hello! Let's test your knowledge about words.
What is the correct definition of this word: ephemeral



Definition #0 is: lasting for a very short time


Definition #1 is: lasting for a long period of time


Definition #2 is: occurring frequently


Definition #3 is: having a strong scent


What is your choice? (Return the single number option)3


In [28]:
answer

'having a strong scent'

## Step 6: Return Local Function Response to the Tool in the Assistant

In [29]:
run = client.beta.threads.runs.submit_tool_outputs(
    thread_id=thread.id,
    run_id=run.id,
    tool_outputs=[
        {
            "tool_call_id": tool_call.id, #tool_call will have unique id
            "output": json.dumps(answer),
        }
    ],
)

In [30]:
run = wait_on_run(run, thread)

In [31]:
run.status

'completed'

## Step 7: Retrieve Messages

In [32]:
messages = client.beta.threads.messages.list(thread.id)

In [33]:
for thread_message in messages:
    print(thread_message.content[0].text.value)
    print('\n')

The student's answer is incorrect. The correct definition of the word "ephemeral" is "lasting for a very short time".


Create a new quiz question word and definition list. Then let me know if the student answer recieved is correct.




## Optional: Delete Assistant

In [34]:
my_assistants = client.beta.assistants.list(
    order="desc",
    limit="20",
)
response = client.beta.assistants.delete(my_assistants.data[0].id)
print(response)

AssistantDeleted(id='asst_Z7g503iYbzrKminpljxjATnk', deleted=True, object='assistant.deleted')
